# World Model Core
*Sean Steinle, Kiya Aminfar*

This notebook walks through the core aspects of world models, developing crucial pieces of code sequentially. Not that this code isn't meant for scale -- instead, this is for a demonstration of how we developed the code that we did.

## Table of Contents
1. [Collecting Rollout Data](#Collecting-Rollout-Data)
2. [Training the VAE](#Training-the-VAE)
3. [Training the MDN-RNN](#Training-the-MDN-RNN)
    - [Prepping Rollout Data for the MDN-RNN](#Prepping-Rollout-Data-for-the-MDN-RNN)
    - [Core Training](#Core-Training)
4. [Training the Controller](#Training-the-Controller)
5. [Early Results](#Early-Results)

In [1]:
import gymnasium as gym
import matplotlib.pyplot as plt
import os
import numpy as np

## Collecting Rollout Data

In [2]:
#Let's begin by creating an instance of our humanoid environment and checking out what basic observations look like.
env = gym.make('Humanoid-v5', render_mode="rgb_array")
obs, info = env.reset()

In [3]:
obs.shape, obs

((348,),
 array([ 1.40407487e+00,  9.91028598e-01,  8.77444272e-03,  5.50217648e-03,
        -2.10420081e-03,  4.36327484e-03, -7.30805570e-03,  2.62785484e-03,
         5.32032757e-03, -2.13750113e-03,  6.26080991e-03,  6.60496846e-03,
        -6.42673052e-03,  9.53574127e-03,  9.72645665e-03, -3.38799013e-03,
         9.68417577e-03, -3.07680594e-03, -3.70274456e-03, -2.88886374e-04,
        -3.86991517e-03, -6.53416455e-03,  8.83760276e-03, -4.37538126e-03,
         2.81940756e-03,  9.35126923e-03,  6.74244336e-03, -9.92979233e-03,
        -5.56664951e-03, -5.03968813e-04,  4.81224281e-03, -5.05962554e-03,
         4.29472192e-03,  5.12513196e-03, -5.20837592e-03,  4.54432324e-03,
        -3.58355926e-03, -8.42856154e-03, -9.10236022e-03,  8.31343887e-05,
         4.87503444e-03, -7.41129833e-05,  3.06770526e-03, -9.35102372e-03,
        -2.69693217e-03,  2.30539831e+00,  2.28693358e+00,  4.38522522e-02,
        -1.21215543e-03,  5.27647113e-02,  4.69547496e-02, -1.18002608e-01,
   

In [4]:
info

{'x_position': np.float64(0.001754087386925571),
 'y_position': np.float64(-0.0008231982206286306),
 'tendon_length': array([-0.01311445,  0.00034416]),
 'tendon_velocity': array([-0.0006738 , -0.01033351]),
 'distance_from_origin': np.float64(0.0019376475095892755)}

As we can see, the humanoid environment gives us a TON of observations! We get dozens of variables representing various positions and velocities of body parts, the center of mass, and a lot of other variables I hardly understand. For an exhaustive list, see the [doc](https://gymnasium.farama.org/environments/mujoco/humanoid/#observation-space). The fact that there are so many variables here is what makes learning latent observations so obvious!

We also get some nice summary stats in info, but we aren't going to include them in our scrape.

In [20]:
def collect_rollout_data(env_name: str, out_dir: str, n_timesteps: int=10000, print_n_episodes: int=1000):
    """Simulates `n_timesteps` in the `env_name` environment, saving observations, rewards, and actions to a triplet of .npy files at `out_dir`."""
    env = gym.make(env_name, render_mode='rgb_array')
    obs, info = env.reset()
    observations, rewards, actions = [], [] , []
    episode_count = 0

    for timestep in range(n_timesteps):  # Run for n_timesteps or until the episode ends
        action = env.action_space.sample() #select random action
        obs, reward, terminated, truncated, info = env.step(action) #execute and get results
        observations.append(obs) #save observation
        rewards.append(reward) #save reward
        actions.append(action) #save action
        if terminated or truncated: #check for game over, if so reset env
            episode_count+=1
            if episode_count % print_n_episodes == 0: print(f"finished {episode_count} episodes") #provide update on training
            observation, info = env.reset()
        env.close()
    np_obs, np_rewards, np_actions = np.array(observations), np.array(rewards), np.array(actions)
    print(f"observations has shape: {np_obs.shape}\trewards has shape: {np_rewards.shape}\tactions has shape: {np_actions.shape}")
    np.save(f'{out_dir}/{env_name}_{n_timesteps}_rollout_observations.npy', np_obs) #load with: new_obs = np.load("../data/processed/Humanoid-v5_10000_rollout_observations.npy")
    np.save(f'{out_dir}/{env_name}_{n_timesteps}_rollout_rewards.npy', np_rewards)
    np.save(f'{out_dir}/{env_name}_{n_timesteps}_rollout_actions.npy', np_actions)
    return np_obs, np_rewards, np_actions

In [22]:
humanoid_obs, humanoid_rewards, humanoid_actions = collect_rollout_data('Humanoid-v5', "../data/processed", 10000, 100)

finished 100 episodes
finished 200 episodes
finished 300 episodes
finished 400 episodes
observations has shape: (10000, 348)	rewards has shape: (10000,)	actions has shape: (10000, 17)


In [23]:
humanoid_rewards

array([4.89014007, 4.92889786, 4.94746227, ..., 4.52836352, 4.57897921,
       4.56725108])

In [24]:
humanoid_actions

array([[-0.1344295 , -0.33341795,  0.2567148 , ..., -0.26523423,
        -0.22723526, -0.34251404],
       [-0.2680746 , -0.05566841, -0.06947935, ..., -0.19162744,
        -0.23866333,  0.33647585],
       [-0.1664092 ,  0.0057418 , -0.32716182, ...,  0.06560624,
         0.14931448,  0.23692013],
       ...,
       [-0.3987542 ,  0.35181522,  0.2205222 , ..., -0.27054495,
         0.26330063, -0.19415343],
       [-0.26650003, -0.35020053, -0.03938405, ..., -0.18459655,
         0.04306056,  0.2507781 ],
       [-0.25835124,  0.19055103, -0.2883535 , ...,  0.04691168,
        -0.00619155, -0.23913088]], dtype=float32)

## Training the VAE

Now that we have an easy function for gathering experiences in the environment, we need to train the VAE module of our world model which will compress the observation space into latent space with fewer dimensions.

Note that the original World Model implementation worked with tensorflow 1.18.0. This is incredibly outdated (worked with Python 3.5), so let's get a newer version (tensorflow 2.19.0). Additionally, we need to change the structure of the VAE from working with images to working with a vector of observation data! Luckily, ChatGPT is very good at updating code (or it will be very obvious if it is not!).

In [7]:
import tensorflow as tf
from tensorflow.keras import layers, Model, saving

@saving.register_keras_serializable()
class MLPVAE(Model):
    def __init__(self, input_dim=348, z_size=32, kl_tolerance=0.5):
        super(MLPVAE, self).__init__()
        self.z_size = z_size
        self.kl_tolerance = kl_tolerance

        # Encoder
        self.encoder = tf.keras.Sequential([
            layers.InputLayer(input_shape=(input_dim,)),
            layers.Dense(256, activation='relu'),
            layers.Dense(128, activation='relu'),
            layers.Dense(2 * z_size),  # output both mu and logvar
        ])

        # Decoder
        self.decoder = tf.keras.Sequential([
            layers.InputLayer(input_shape=(z_size,)),
            layers.Dense(128, activation='relu'),
            layers.Dense(256, activation='relu'),
            layers.Dense(input_dim, activation='linear'),  # output same shape as input
        ])

    def sample_z(self, mu, logvar):
        eps = tf.random.normal(shape=tf.shape(mu))
        sigma = tf.exp(0.5 * logvar)
        return mu + sigma * eps

    def encode(self, x):
        h = self.encoder(x)
        mu, logvar = tf.split(h, num_or_size_splits=2, axis=1)
        logvar = tf.clip_by_value(logvar, -10.0, 10.0)  # helps with exploding values
        return mu, logvar

    def decode(self, z):
        return self.decoder(z)

    def call(self, x):
        mu, logvar = self.encode(x)
        z = self.sample_z(mu, logvar)
        x_recon = self.decode(z)
        return x_recon, mu, logvar

    def compute_loss(self, x):
        x_recon, mu, logvar = self(x)
        recon_loss = tf.reduce_mean(tf.reduce_sum(tf.square(x - x_recon), axis=1))
        kl_loss = -0.5 * tf.reduce_sum(1 + logvar - tf.square(mu) - tf.exp(logvar), axis=1)
        kl_loss = tf.maximum(kl_loss, self.kl_tolerance * self.z_size)
        kl_loss = tf.reduce_mean(kl_loss)
        total_loss = recon_loss + kl_loss
        return total_loss, recon_loss, kl_loss

2025-05-07 12:59:12.890722: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-05-07 12:59:12.911558: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746637152.932483   18052 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746637152.939156   18052 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1746637152.953001   18052 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

Let's also write a training function for convenience.

In [8]:
def create_dataset(x_train, batch_size=64, shuffle_buffer=10000):
    # Assuming x_train is a NumPy array of shape [n_samples, 348]
    dataset = tf.data.Dataset.from_tensor_slices(x_train.astype(np.float32))
    dataset = dataset.shuffle(shuffle_buffer).batch(batch_size).prefetch(tf.data.AUTOTUNE)
    return dataset

def train_vae(model, dataset, epochs=10, learning_rate=1e-4):
    optimizer = tf.keras.optimizers.Adam(learning_rate)

    for epoch in range(epochs):
        total_loss = 0.0
        total_batches = 0
        for x_batch in dataset:
            with tf.GradientTape() as tape:
                loss, recon_loss, kl_loss = model.compute_loss(x_batch)
            grads = tape.gradient(loss, model.trainable_variables)
            optimizer.apply_gradients(zip(grads, model.trainable_variables))

            total_loss += loss.numpy()
            total_batches += 1

        avg_loss = total_loss / total_batches
        print(f"Epoch {epoch+1}: avg loss = {avg_loss:.4f}")


In [9]:
# x_train should be a NumPy array of shape (n_samples, 348)
x_train = humanoid_obs
x_train = (x_train - np.mean(x_train, axis=0)) / (np.std(x_train, axis=0) + 1e-6)

dataset = create_dataset(humanoid_obs, batch_size=64)
vae = MLPVAE(input_dim=348, z_size=32)
train_vae(vae, dataset, epochs=20)

2025-05-07 12:59:18.962595: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)
/home/seansteinle/.pyenv/versions/cs757-final-project/lib/python3.10/site-packages/keras/src/layers/core/input_layer.py:27: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(
2025-05-07 12:59:39.739753: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Epoch 1: avg loss = 300894.9688


2025-05-07 13:00:10.633088: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Epoch 2: avg loss = 142766.4375
Epoch 3: avg loss = 67727.7812


2025-05-07 13:00:54.702790: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Epoch 4: avg loss = 56640.4141
Epoch 5: avg loss = 51510.5586
Epoch 6: avg loss = 46468.8984
Epoch 7: avg loss = 41013.7070


2025-05-07 13:02:34.166434: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Epoch 8: avg loss = 35455.9766
Epoch 9: avg loss = 30666.1250
Epoch 10: avg loss = 26413.6094
Epoch 11: avg loss = 23376.8887
Epoch 12: avg loss = 21192.2285
Epoch 13: avg loss = 19393.9473
Epoch 14: avg loss = 17780.1152
Epoch 15: avg loss = 16335.0908


2025-05-07 13:04:56.819597: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Epoch 16: avg loss = 14862.1846
Epoch 17: avg loss = 13538.6533
Epoch 18: avg loss = 12628.1885
Epoch 19: avg loss = 11968.4492
Epoch 20: avg loss = 11390.0234


Loss is going down! At first I got a tons of NAN values, but it's because I wasn't normalizing the input data and I also needed to clip the logvar values we were getting as a result of the encoding process. If you get NANs again, a lower learning rate could help too. Onto saving the model!

In [10]:
vae.save_weights('../models/vae/humanoid_10000_vae_model.weights.h5') #save ONLY weights -- much simpler than serializing the entire object

In [11]:
new_vae = MLPVAE(input_dim=348, z_size=32) #instantiate new model object 
new_vae(tf.zeros((1, 348))) #invoke it to build its shape 
new_vae.load_weights('../models/vae/humanoid_10000_vae_model.weights.h5') #now load weights into empty vector

In [12]:
new_vae

<MLPVAE name=mlpvae_1, built=True>

In [13]:
vae

<MLPVAE name=mlpvae, built=True>

## Training the MDN-RNN

Now that we have a model which captures observations, we're theoretically ~1/3 done with the project! I say theoretically because this was probably the easiest part of the project. Now onto the meat of world models: capturing the transitions of our environment and training the MDN-RNN!

### Prepping Rollout Data for the MDN-RNN

To train the MDN-RNN, we first need to enhance our basic rollout dataset with predictions of `mu` and `logvar` for each experience. Then we'll feed this information to the MDN-RNN.

In [30]:
#We can use the dataset records still in memory. Note that we only need the observations for now!
humanoid_obs.shape, humanoid_rewards.shape, humanoid_actions.shape

((10000, 348), (10000,), (10000, 17))

In [32]:
#We can also use the VAE still in memory!
vae

<MLPVAE name=mlpvae, built=True>

In [39]:
humanoid_obs[0]

array([ 1.40143877e+00,  9.99960057e-01,  2.13178531e-03,  1.76368766e-03,
        8.49880557e-03, -5.03831888e-02,  1.99407773e-02,  1.06239878e-03,
       -3.86322047e-03, -7.36627872e-03, -1.12246644e-01, -1.69467332e-01,
       -4.09778076e-03,  7.37100435e-02, -4.60652693e-02, -7.12577640e-02,
       -2.35541353e-02, -2.76335521e-02,  1.99025827e-02, -1.73195788e-02,
       -2.97512321e-02, -3.05629178e-02, -4.52620978e-02,  4.35240471e-02,
       -2.65623041e-01, -4.93006524e-01,  9.99974414e-01,  1.97973186e+00,
       -6.20208208e+00,  3.82205018e-01,  1.21927149e+00, -4.56408217e-01,
       -8.82785313e-01, -1.14253583e+01, -2.06736930e+01,  1.39722530e-01,
        6.13995974e+00, -4.24498038e+00, -7.70727524e+00, -2.25350381e+00,
       -1.46339718e+00,  1.93039896e+00, -6.63493524e-01, -3.99338732e+00,
       -3.35115979e+00,  2.29871023e+00,  2.28197984e+00,  4.36726832e-02,
        7.15717279e-05,  6.74920052e-02,  8.43825106e-03, -1.46284824e-01,
       -1.64198951e-02,  

In [40]:
#That's it! Just predict for our entire observation set.
mu, logvar = vae.encode(humanoid_obs)
mu.shape, logvar.shape #makes sense -- a mean and stdev for each dimen of latent space for each sample

(TensorShape([10000, 32]), TensorShape([10000, 32]))

In [44]:
np.save(f'../data/processed/Humanoid-v5_10000_rollout_mu.npy', mu)
np.save(f'../data/processed/Humanoid-v5_10000_rollout_logvar.npy', logvar)

In [ ]:
#The 'recipe' is missing task hierarchies!
    #We never cared before because mastering domains was infeasible.

#Minecraft paper offers a cool example because we have a very clear task hierarchy! But crafting task hierarchies in real environments are nontrivial -- @Nick.

#Is foundation models as a prior good enough?
#We don't just need information about what environments are like, we need goals! We need to know what it takes to be good at X.
    #This is a RICH area for assurance research too. This set of goals is necessarily broad, but need to ensure they align tightly to our values and domain mastery.
#Information about computers might help with tasks, but what are the tasks? And how do the tasks present a cohesive picture of domain mastery?